In [1]:
import autogen
import os 
import json
import dotenv
from pathlib import Path
from typing import Tuple, Optional
import shutil
import httpx


os.environ["HTTPX_TIMEOUT"] = "300"  # 5 minutes

# llm configs
dotenv.load_dotenv('.env')
openai_api_key = os.getenv('OPENAI_API_KEY')

GPT4_CONFIG = {
    "model": "gpt-4",
    "api_key": openai_api_key,
}

GPT5_CONFIG = {
    "model": "gpt-5-mini-2025-08-07",
    "api_key": openai_api_key,
}

# autogen configs
llm_config_gpt4 = {
    "seed": 30,
    "config_list": [GPT4_CONFIG],
    "request_timeout": 600,
    "retry_wait_time": 120
}
llm_config_gpt5 = {
    "seed": 30,
    "config_list": [GPT5_CONFIG],
    "request_timeout": 600,
    "retry_wait_time": 120
}

# workspace
WORKSPACE = Path("fem_workspace")
WORKSPACE.mkdir(exist_ok=True)
for subdir in ["mesh", "config", "solution", "analysis", "strategies"]:
    (WORKSPACE / subdir).mkdir(exist_ok=True) 


## ========================================
## VALIDATORS (UPDATED - GENERIC APPROACH)
## ========================================
class Validator:
    '''Generic validators using pattern detection'''

    @staticmethod
    def validate_mesh(workspace: Path) -> Tuple[bool, list]:
        """Validate mesh was generated with acceptable quality"""
        issues = []

        mesh_file = workspace / "mesh" / "mesh.msh"
        marker_map_file = workspace / "mesh" / "marker_map.json"
        quality_file = workspace / "mesh" / "mesh_quality.json"
        viz_file = workspace / "mesh" / "mesh_visualization.png"

        if not mesh_file.exists():
            issues.append("mesh.msh file not found.")
            return False, issues
        if not marker_map_file.exists():
            issues.append("marker_map.json file not found (required for boundary markers).")
            return False, issues
        if not quality_file.exists():
            issues.append("mesh_quality.json file not found.")
            return False, issues
        if not viz_file.exists():
            issues.append("mesh_visualization.png file not found.")
            return False, issues

        # Check PNG is not empty
        if viz_file.exists() and viz_file.stat().st_size < 1000:
            issues.append(f"mesh_visualization.png is too small ({viz_file.stat().st_size} bytes)")
            
        # Check quality metric
        try:
            with open(quality_file) as f:
                quality = json.load(f)
            
            # Check meshing method
            method = quality.get("meshing_method", "unknown")
            if method == "block_structured_partitioning":
                num_blocks = quality.get("num_blocks", 0)
                if num_blocks > 0:
                    print(f"    ✓ Partitioned mesh: {num_blocks} blocks")
            elif method == "frontal_delaunay_fallback":
                print(f"    ℹ️ Mesh used fallback (Algorithm 8)")

            if quality.get("num_elements", 0) == 0:
                issues.append("0 elements in the mesh.")
                return False, issues
            
            # Get reported quality
            min_q = quality.get("min_quality") or quality.get("min_quality_proxy") or 0
            
            # FIX: If quality is 0.0, recompute using correct Gmsh API
            if min_q == 0.0:
                print("    ⚠️  Quality reported as 0.0 - recomputing with Gmsh API...")
                recomputed_min, recomputed_avg = Validator._recompute_mesh_quality(mesh_file)
                
                if recomputed_min is not None:
                    min_q = recomputed_min
                    print(f"    ✓ Recomputed quality: min={recomputed_min:.4f}, avg={recomputed_avg:.4f}")
                    
                    # Update the JSON file with correct values
                    quality["min_quality"] = round(recomputed_min, 4)
                    quality["avg_quality"] = round(recomputed_avg, 4)
                    quality["fenics_compatible"] = True
                    with open(quality_file, "w") as f:
                        json.dump(quality, f, indent=2)
                else:
                    print("    ✗ Could not recompute quality")
            
            if min_q < 0.2:
                issues.append(f"low quality: {min_q:.2f}")
            
            # Verify mesh is marked as ready
            if not quality.get("mesh_ready_for_solver", False):
                issues.append("Mesh not marked as ready for solver")
                
        except Exception as e:
            issues.append(f"Error reading mesh quality: {e}")
            
        return len(issues) == 0, issues

    @staticmethod
    def _recompute_mesh_quality(mesh_file: Path) -> Tuple[Optional[float], Optional[float]]:
        """Recompute mesh quality using correct Gmsh 4.15.0 API"""
        try:
            import gmsh
            import numpy as np
            
            gmsh.initialize()
            gmsh.option.setNumber("General.Terminal", 0)  # Suppress output
            gmsh.open(str(mesh_file))
            
            # Get element tags
            elem_types, elem_tags, _ = gmsh.model.mesh.getElements(dim=2)
            all_tags = []
            for tags in elem_tags:
                all_tags.extend(tags)
            
            if len(all_tags) == 0:
                gmsh.finalize()
                return None, None
            
            # Correct API: getElementQualities (plural) with tags
            qualities = gmsh.model.mesh.getElementQualities(all_tags)
            
            gmsh.finalize()
            
            min_q = float(np.min(qualities))
            avg_q = float(np.mean(qualities))
            
            return min_q, avg_q
            
        except Exception as e:
            print(f"    ✗ Quality recomputation failed: {e}")
            try:
                gmsh.finalize()
            except:
                pass
            return None, None
    
    @staticmethod
    def validate_config(workspace: Path) -> Tuple[bool, list]:
        """Basic config file existence check"""
        issues = []

        config_file = workspace / "config" / "problem_config.json"
        
        if not config_file.exists():
            issues.append("problem_config.json file not found.")
            return False, issues
        
        try:
            with open(config_file) as f:
                config = json.load(f)
            
            if len(config) == 0:
                issues.append("Config file is empty.")
                
        except Exception as e:
            issues.append(f"Error reading problem config: {e}")
        
        return len(issues) == 0, issues
    
    @staticmethod
    def validate_formulation_generic(workspace: Path) -> Tuple[bool, list, list]:
        """Generic formulation validation using pattern detection"""
        errors = []
        warnings = []
        
        config_file = workspace / "config" / "problem_config.json"
        
        if not config_file.exists():
            errors.append({"type": "missing_file", "msg": "problem_config.json not found"})
            return False, errors, []
        
        try:
            with open(config_file) as f:
                config = json.load(f)
            
            # PATTERN 1: Detect singular dict that should be a list
            for key, value in config.items():
                if isinstance(value, dict):
                    hint_words = ["value", "region", "marker", "location", "component", "dof"]
                    if any(hint in value for hint in hint_words):
                        if not key.endswith("s"):
                            plural_key = key + "s"
                            warnings.append({
                                "type": "singular_to_plural",
                                "field": key,
                                "plural_field": plural_key,
                                "suggestion": f"Field '{key}' contains BC/load dict but isn't a list. Solvers often expect '{plural_key}' as a list.",
                                "fixable": True
                            })
            
            # PATTERN 2: Check for suspiciously zero material properties
            zero_props = []
            for key, value in config.items():
                if isinstance(value, (int, float)) and value == 0:
                    if key.upper() == key or key in ["E", "G", "K", "nu", "rho", "cp", "lambda", "alpha"]:
                        zero_props.append(key)
            
            if zero_props:
                warnings.append({
                    "type": "zero_property",
                    "fields": zero_props,
                    "suggestion": f"Material properties {zero_props} are zero - might be uninitialized",
                    "fixable": False
                })
            
            # PATTERN 3: Check for empty lists where content expected
            for key, value in config.items():
                if isinstance(value, list) and len(value) == 0:
                    if any(word in key.lower() for word in ["bc", "boundary", "load", "traction", "force"]):
                        warnings.append({
                            "type": "empty_list",
                            "field": key,
                            "suggestion": f"Field '{key}' is an empty list - no conditions defined?",
                            "fixable": False
                        })
        
        except Exception as e:
            errors.append({"type": "read_error", "msg": f"Error reading config: {e}"})
        
        is_valid = len(errors) == 0
        return is_valid, errors, warnings
    
    @staticmethod
    def auto_fix_formulation(workspace: Path, warnings: list) -> Tuple[bool, list]:
        """Automatically fix common formulation issues based on patterns"""
        config_file = workspace / "config" / "problem_config.json"
        
        with open(config_file) as f:
            config = json.load(f)
        
        fixes = []
        
        for warning in warnings:
            if not warning.get("fixable", False):
                continue
            
            if warning["type"] == "singular_to_plural":
                key = warning["field"]
                plural_key = warning["plural_field"]
                
                if key in config:
                    value = config.pop(key)
                    config[plural_key] = [value]
                    fixes.append(f"Converted '{key}' → '{plural_key}' (wrapped dict in list)")
        
        if fixes:
            with open(config_file, "w") as f:
                json.dump(config, f, indent=2)
        
        return len(fixes) > 0, fixes
    
    @staticmethod
    def validate_solution(workspace: Path) -> Tuple[bool, list]:
        """Validate solution with basic physics sanity checks"""
        issues = []
        
        metrics_file = workspace / "solution" / "solver_metrics.json"
        config_file = workspace / "config" / "problem_config.json"
        
        if not metrics_file.exists():
            issues.append("solver_metrics.json missing")
            return False, issues
        
        try:
            with open(metrics_file) as f:
                metrics = json.load(f)
            
            if not metrics.get("converged", False):
                issues.append("Solver did not converge")
                return False, issues
            
            with open(config_file) as f:
                config = json.load(f)
            
            has_forcing = False
            forcing_indicators = ["traction", "tractions", "load", "loads", "force", "forces", 
                                 "body_force", "source", "heat_source", "current", "pressure"]
            
            for key in config.keys():
                if any(indicator in key.lower() for indicator in forcing_indicators):
                    value = config[key]
                    if isinstance(value, (int, float)) and value != 0:
                        has_forcing = True
                        break
                    elif isinstance(value, (list, dict)) and value:
                        has_forcing = True
                        break
            
            if has_forcing:
                zero_count = 0
                total_metrics = 0
                
                for key, value in metrics.items():
                    if "max_" in key or "mean_" in key:
                        total_metrics += 1
                        if isinstance(value, (int, float)) and value == 0.0:
                            zero_count += 1
                
                if total_metrics > 0 and zero_count == total_metrics:
                    issues.append("PHYSICS WARNING: All solution metrics are zero despite applied forcing")
                
        except Exception as e:
            issues.append(f"Error in solution validation: {e}")
        
        return len(issues) == 0, issues


# ========================================
# PROMPTS
# ========================================

DECOMPOSER_PROMPT = """You are an expert at analyzing FEM problems.
Analyze the problem and create a detailed structured plan (JSON only).
CRITICAL: All numeric values in JSON must be literal numbers, NOT arithmetic expressions.

STAGE DEPENDENCIES AND OUTPUTS:
- Stage 1 (Meshing): Creates mesh/mesh.msh, mesh/marker_map.json, mesh/mesh_visualization.png, mesh/mesh_quality.json
- Stage 2 (Formulation): Creates config/problem_config.json
- Stage 3 (Solving): Reads from mesh/ and config/, creates solution/displacement.xdmf, solution/stress.xdmf, solution/solver_metrics.json, solution/*.png
- Stage 4 (Analysis): Reads from solution/, config/, mesh/, creates analysis/analysis_report.json, analysis/summary.txt

{
  "problem_summary": "brief description",
  "stages": [
    {
      "stage_id": 1,
      "name": "Meshing",
      "description": "Create quad mesh with boundary markers",
      "geometry_analysis": {
        "outer_boundary": "shape and dimensions",
        "internal_features": ["holes, cutouts, etc."],
        "singular_points": ["corners, stress concentrations"]
      },
      "strategy_files": [
        "strategies/README.md",
        "strategies/geometry_patterns/<relevant_pattern>.md",
        "strategies/quad_meshing/PARTITIONING_WORKFLOW.md"
      ],
      "refinement_zones": ["areas needing fine mesh"],
      "outputs": ["mesh/mesh.msh", "mesh/marker_map.json", "mesh/mesh_visualization.png", "mesh/mesh_quality.json"]
    },
    {
      "stage_id": 2,
      "name": "Formulation",
      "description": "Define problem configuration",
      "material_properties": {"E": "value in Pa", "nu": "value"},
      "boundary_conditions": {"name": {"type": "...", "location": "..."}},
      "outputs": ["config/problem_config.json"]
    },
    {
      "stage_id": 3,
      "name": "Solving",
      "description": "Load mesh from mesh/mesh.msh and config from config/problem_config.json. Solve FEM problem, compute stress/displacement, create visualizations. DO NOT regenerate mesh.",
      "inputs": ["mesh/mesh.msh", "mesh/marker_map.json", "config/problem_config.json"],
      "solver_guidance": {"solver": "direct or iterative", "degree": 1 or 2},
      "outputs": ["solution/displacement.xdmf", "solution/stress.xdmf", "solution/solver_metrics.json", "solution/displacement.png", "solution/sigma_xx.png"]
    },
    {
      "stage_id": 4,
      "name": "Analysis",
      "description": "Read all solver outputs, provide engineering-level interpretation and insights",
      "inputs": ["solution/*", "config/problem_config.json", "mesh/mesh_quality.json"],
      "outputs": ["analysis/analysis_report.json", "analysis/summary.txt"]
    }
  ]
}

AVAILABLE STRATEGY FILES FOR MESHING:
- strategies/README.md (decision tree for meshing approach)
- strategies/geometry_patterns/plate_with_hole.md
- strategies/geometry_patterns/rectangle.md
- strategies/quad_meshing/PARTITIONING_WORKFLOW.md

MESHING APPROACH RULES:
- Rectangle (no hole): Transfinite (1 block)
- Rectangle + hole: OCC boolean (addRectangle + addDisk + cut) + Algorithm 8
- L-shape or complex: OCC boolean + Algorithm 8
- DO NOT request manual O-grid block construction for geometries with holes

CRITICAL RULES:
- Solver stage must NEVER regenerate mesh - it uses existing mesh/mesh.msh
- Analysis stage interprets results and provides engineering insights
- All file paths are relative to fem_workspace/

Output ONLY JSON."""


MESHING_PROMPT = """You are a mesh generation expert using Gmsh Python API.

## MANDATORY FIRST STEP
Before writing ANY code, read the strategy files provided, in the /fem_workspace/strategies directory in your task and copy the code patterns exactly - do not invent your own Gmsh API calls
Start with strategies/README.md, then read the specific files that can help you best solve the problem from strategies/geometry_patterns/, and finally read strategies/quad_meshing/ files and apply the most suitable approach.

## Your Process (Document EACH step before coding)

### STEP 1: STATE THE GEOMETRY
Write: outer boundary shape, dimensions, holes, features.

### STEP 2: IDENTIFY KEY FEATURES
List: corners, hole centers, re-entrant corners, stress concentration areas.

### STEP 3: CHOOSE MESHING APPROACH
Based on the strategy files, decide:
- Which Gmsh kernel to use (geo vs occ)
- Which meshing algorithm
- Block structure (if applicable)

### STEP 4: PLAN REFINEMENT
Identify areas needing finer mesh (near holes, corners, load application points).

### STEP 5: PLAN BOUNDARY MARKERS
List physical groups needed for boundary conditions:
- Which edges need markers (left, right, top, bottom, hole, etc.)
- Marker IDs to assign

### STEP 6: WRITE CODE
Implement following the strategy files and your documented plan.

## REQUIRED OUTPUTS

| File | Format | Purpose |
|------|--------|---------|
| mesh/mesh.msh | Gmsh MSH format | Main mesh for DOLFINx solver (read via gmsh) |
| mesh/marker_map.json | JSON | Maps boundary names to integer tags |
| mesh/mesh_visualization.png | PNG image | Visual verification of mesh |
| mesh/mesh_quality.json | JSON | Quality metrics for validation |

## EXPORT REQUIREMENTS
- Keep the .msh file (DO NOT convert to XDMF)
- Solver will read .msh directly using dolfinx.io.gmsh
- Write marker_map.json mapping boundary names to physical group IDs

## marker_map.json FORMAT
```json
{
  "<boundary_name>": <int>,
  "<boundary_name>": <int>,
  ...
  "domain": <int>
}
```

RULES:
- Use boundary names that match the problem description (e.g., "left_edge", "right", "fixed_support")
- Each boundary gets a unique integer tag (starting from 1)
- The "domain" entry marks the 2D surface (use a higher number like 100)
- Only include boundaries that EXIST in the geometry

EXAMPLES BY GEOMETRY TYPE:

Simple rectangle (no holes):
```json
{
  "left": 1,
  "right": 2,
  "top": 3,
  "bottom": 4,
  "domain": 100
}
```

Plate with one hole:
```json
{
  "left_edge": 1,
  "right_edge": 2,
  "top_edge": 3,
  "bottom_edge": 4,
  "hole_boundary": 5,
  "domain": 100
}
```

Plate with multiple holes:
```json
{
  "left": 1,
  "right": 2,
  "top": 3,
  "bottom": 4,
  "hole_1": 5,
  "hole_2": 6,
    .
    .
//and so on
  "domain": 100
}
```

L-shape or notched geometry:
```json
{
  "outer_left": 1,
  "outer_bottom": 2,
  "inner_vertical": 3,
  "inner_horizontal": 4,
  "outer_right": 5,
  "outer_top": 6,
  "domain": 100
}
```

Use the SAME marker names that appear in the problem description for boundary conditions.

## mesh_quality.json EXACT FORMAT (DO NOT MODIFY FIELD NAMES)
- num_elements: <int>
- element_type: "quad" or "triangle"
- min_quality: <float>
- avg_quality: <float>
- mesh_ready_for_solver: <bool>
- fenics_compatible: <bool>

## QUALITY EXTRACTION (CRITICAL)
- `gmsh.model.mesh.getElementQualities(element_tags)` - note PLURAL and requires tags

## CODE REQUIREMENTS
1. Use ```python code blocks
2. Include: # filename: generate_mesh.py at the top
3. State your reasoning BEFORE the code (2-3 sentences)
4. Print confirmation that mesh is ready for solver
5. Generate and save a PNG visualization of the mesh
6. After successful execution, reply: TERMINATE

## COMMON MISTAKES TO AVOID
- Using `getElementQuality()` instead of `getElementQualities(tags)`
- Using `from dolfinx.io import gmshio` instead of `from dolfinx.io import gmsh`
- Writing quality as 0.0 when extraction fails (debug instead!)

Follow the strategy files. The solver agent depends on your mesh being correct."""


FORMULATION_PROMPT = """You are a FEM problem formulation expert.

Create config/problem_config.json with all information the solver needs.

## EXACT SCHEMA (USE THESE EXACT KEY NAMES)
```json
{
  "formulation_rationale": "1-2 sentences explaining key decisions",
  
  "E": <float in Pa>,
  "nu": <float>,
  "plane_state": "plane_stress" or "plane_strain",
  
  "dirichlet_bcs": [
    {
      "marker_name": "<string: MUST match a key in marker_map.json, e.g. 'left', 'right'>",
      "component": "ux" or "uy" or "all",
      "value": <float>
    }
  ],
  
  "tractions": [
    {
      "marker_name": "<string: MUST match a key in marker_map.json>",
      "traction_vector_Pa": [<float tx>, <float ty>]
    }
  ],
  
  "body_force": [<float fx>, <float fy>],
  "element_degree": <int: 1 or 2>
}
```

CRITICAL RULES:
- Use EXACTLY these key names: "dirichlet_bcs", "tractions", "marker_name", "traction_vector_Pa"
- "marker_name" MUST match keys from mesh/marker_map.json (e.g., "left", "right", "hole")
- E and nu at TOP LEVEL (not nested under "material")
- Do NOT use: "neumann_tractions", "boundary_conditions", "markers", "dofs"
- ALWAYS include 'formulation_rationale' as the FIRST field
- If there can be MULTIPLE BCs or tractions → use a LIST (even if only one entry)
- Units: SI (Pa, meters)

Output ONLY the JSON, then say "TERMINATE"."""


SOLVER_PROMPT = """You are a FEM solver expert using DOLFINx 0.10.0 (FEniCSx).

## DOLFINX 0.10.0 API - CRITICAL PATTERNS

### Imports
```python
from mpi4py import MPI
from petsc4py import PETSc
import numpy as np
import ufl

from dolfinx import fem, io, mesh, plot, default_scalar_type
from dolfinx.fem import (
    functionspace, Function, dirichletbc, 
    locate_dofs_topological, locate_dofs_geometrical,
    form, Constant, Expression
)
from dolfinx.fem.petsc import LinearProblem  # Explicit import required
from dolfinx.mesh import locate_entities_boundary, meshtags
```

### Function Space (Vector for elasticity)
```python
# 2D elasticity
V = functionspace(domain, ("Lagrange", degree, (domain.geometry.dim,)))

# Scalar field (for stress projection)
W = functionspace(domain, ("DG", 0))  # or ("Lagrange", 1)
```

### Loading Mesh from MSH (Gmsh format)
```python
from dolfinx.io import gmsh
import json

# Read mesh - handle DOLFINx 0.10.0 return format
result = gmsh.read_from_msh("mesh/mesh.msh", MPI.COMM_WORLD, gdim=2)
domain, cell_tags, facet_tags = result[0], result[1], result[2]

# Load marker name mapping
with open("mesh/marker_map.json") as f:
    marker_map = json.load(f)

# Example: get tag for a boundary
# right_tag = marker_map["right_edge"]
```

### Boundary Conditions
```python
# Find boundary facets
def left_boundary(x):
    return np.isclose(x[0], x_min)

facets = locate_entities_boundary(domain, domain.topology.dim-1, left_boundary)
dofs = locate_dofs_topological(V, domain.topology.dim-1, facets)

# For component-wise BC on vector space
dofs_x = locate_dofs_topological(V.sub(0), domain.topology.dim-1, facets)
bc_ux = dirichletbc(default_scalar_type(0.0), dofs_x, V.sub(0))
```

### Parsing Config (EXACT FORMAT)
```python
with open("config/problem_config.json") as f:
    cfg = json.load(f)

# Material properties (always top-level)
E = float(cfg["E"])
nu = float(cfg["nu"])

# Dirichlet BCs (always "dirichlet_bcs" list)
for bc in cfg.get("dirichlet_bcs", []):
    marker_name = bc["marker_name"]  # matches marker_map.json key
    component = bc["component"]       # "ux", "uy", or "all"
    value = bc["value"]
    
    tag = marker_map.get(marker_name)
    if tag is None:
        continue
    facet_indices = facet_tags.find(tag)
    
    if component == "ux":
        dofs = locate_dofs_topological(V.sub(0), domain.topology.dim-1, facet_indices)
        bcs.append(dirichletbc(default_scalar_type(value), dofs, V.sub(0)))
    elif component == "uy":
        dofs = locate_dofs_topological(V.sub(1), domain.topology.dim-1, facet_indices)
        bcs.append(dirichletbc(default_scalar_type(value), dofs, V.sub(1)))
    elif component == "all":
        for i in range(domain.geometry.dim):
            dofs = locate_dofs_topological(V.sub(i), domain.topology.dim-1, facet_indices)
            bcs.append(dirichletbc(default_scalar_type(value), dofs, V.sub(i)))

# Tractions (always "tractions" list with "traction_vector_Pa")
for t in cfg.get("tractions", []):
    marker_name = t["marker_name"]
    traction_vec = t["traction_vector_Pa"]  # [tx, ty]
    tag = marker_map.get(marker_name)
    # Use in variational form: L += inner(Constant(domain, traction_vec), v) * ds(tag)
```

### Solving with LinearProblem
```python
problem = LinearProblem(
    a, L, 
    bcs=bcs,
    petsc_options={"ksp_type": "preonly", "pc_type": "lu"},
    petsc_options_prefix="solve_"  # REQUIRED in 0.10.0, must be NON-EMPTY
)
uh = problem.solve()
uh.x.scatter_forward()  # Update ghost values
```

### Computing and Projecting Stress
```python
# Stress tensor from solution
sigma_sol = sigma(uh, lambda_, mu)

# Extract sigma_xx component
sigma_xx_expr = sigma_sol[0, 0]

# Project to scalar space
W = functionspace(domain, ("DG", 0))
sigma_xx_func = Function(W)
expr = Expression(sigma_xx_expr, W.element.interpolation_points)
sigma_xx_func.interpolate(expr)
```

### Saving Results to XDMF
NOTE: XDMF writer requires function degree to match mesh degree (usually 1).
For higher-degree solutions, interpolate to CG1 first:
```python
# Interpolate degree-2 displacement to degree-1 for XDMF output
V_out = functionspace(domain, ("Lagrange", 1, (domain.geometry.dim,)))
uh_out = Function(V_out)
uh_out.interpolate(uh)

with io.XDMFFile(MPI.COMM_WORLD, "solution/displacement.xdmf", "w") as xdmf:
    xdmf.write_mesh(domain)
    xdmf.write_function(uh_out)

# For scalar stress (DG0 or higher degree), interpolate to CG1
W_out = functionspace(domain, ("Lagrange", 1))
sigma_out = Function(W_out)
expr = Expression(sigma_xx_expr, W_out.element.interpolation_points)
sigma_out.interpolate(expr)

with io.XDMFFile(MPI.COMM_WORLD, "solution/stress.xdmf", "w") as xdmf:
    xdmf.write_mesh(domain)
    xdmf.write_function(sigma_out)
```

### Visualization with PyVista
```python
import pyvista
from dolfinx.plot import vtk_mesh

pyvista.OFF_SCREEN = True

# For displacement
topology, cell_types, geometry = vtk_mesh(V)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)
grid.point_data["displacement"] = uh.x.array.reshape(-1, domain.geometry.dim)

# Plot
plotter = pyvista.Plotter(off_screen=True)
plotter.add_mesh(grid, scalars="displacement", component=0)
plotter.screenshot("solution/displacement.png")
plotter.close()
```

## INPUTS (created by previous stages)
- mesh/mesh.msh - Mesh file (Gmsh format)
- mesh/marker_map.json - Boundary name to tag mapping
- config/problem_config.json - Problem formulation

## REQUIRED OUTPUTS (all in solution/ folder)

| File | Purpose |
|------|---------|
| solution/displacement.xdmf | Displacement field |
| solution/stress.xdmf | Stress field |
| solution/solver_metrics.json | Results summary |
| solution/displacement.png | Displacement contour |
| solution/sigma_xx.png | σxx stress contour |

## solver_metrics.json FORMAT
- solver_rationale: 1-2 sentences explaining solver choices
- solver_type: "direct" or "iterative"
- element_degree: int
- converged: bool
- max_displacement: float
- max_stress: float

## CODE REQUIREMENTS
1. Use ```python code blocks
2. Include: # filename: solve.py at the top
3. Create solution/ directory: `os.makedirs("solution", exist_ok=True)`
4. State reasoning before code
5. After successful execution, reply: TERMINATE

DO NOT regenerate mesh. Use existing mesh from mesh/mesh.msh."""


ANALYSIS_PROMPT = """You are an FEM analysis expert providing engineering-level insights.

## INPUTS (all files in solution/ folder created by solver)
Read ALL files in the solution/ folder:
- solution/displacement.xdmf - Displacement field data
- solution/stress.xdmf - Stress field data
- solution/solver_metrics.json - Solver results
- solution/displacement.png - Displacement visualization
- solution/sigma_xx.png - Stress visualization
- Any other files the solver created

Also reference:
- config/problem_config.json - Original problem definition
- mesh/mesh_quality.json - Mesh information

## YOUR TASKS
1. Load and examine ALL solver outputs from solution/
2. Analyze displacement field (magnitude, direction, max location)
3. Analyze stress field (concentrations, max/min locations, patterns)
4. Compare to analytical solutions if available (e.g., stress concentration factor for plate with hole: K ≈ 3)
5. Assess mesh adequacy based on results
6. Provide engineering-level insights and recommendations

## REQUIRED OUTPUTS (in analysis/ folder)

| File | Purpose |
|------|---------|
| analysis/analysis_report.json | Comprehensive findings |
| analysis/summary.txt | Human-readable summary |

## analysis_report.json FORMAT
- problem_summary: brief description of what was solved
- displacement_analysis:
  - max_magnitude: float
  - max_location: [x, y]
  - deformation_pattern: description
- stress_analysis:
  - max_sigma_xx: float
  - max_stress_location: [x, y]
  - stress_concentration_factor: float (if applicable)
  - critical_regions: list of descriptions
- comparison_to_theory:
  - expected_value: float (if analytical solution exists)
  - computed_value: float
  - percent_error: float
- mesh_assessment: is mesh adequate for capturing stress concentrations?
- recommendations: list of suggestions (mesh refinement, further analysis, etc.)
- conclusions: engineering summary

## CODE REQUIREMENTS
1. Use ```python code blocks with # filename: analyze.py
2. Create analysis/ directory if it doesn't exist
3. Load data from solution/ folder
4. Print key findings to console
5. After successful execution, reply: TERMINATE

Provide professional engineering interpretation of the FEM results."""


# ========================================
# AGENTS
# ========================================

decomposer = autogen.AssistantAgent(
    name="decomposer",
    system_message=DECOMPOSER_PROMPT,
    llm_config=llm_config_gpt5
)

meshing_specialist = autogen.AssistantAgent(
    name="meshing_specialist",
    system_message=MESHING_PROMPT,
    llm_config=llm_config_gpt5
)

formulation_specialist = autogen.AssistantAgent(
    name="formulation_specialist",
    system_message=FORMULATION_PROMPT,
    llm_config=llm_config_gpt5
)

solver_specialist = autogen.AssistantAgent(
    name="solver_specialist",
    system_message=SOLVER_PROMPT,
    llm_config=llm_config_gpt5
)

analysis_specialist = autogen.AssistantAgent(
    name="analysis_specialist",
    system_message=ANALYSIS_PROMPT,
    llm_config=llm_config_gpt5
)

# Executors
mesh_executor = autogen.UserProxyAgent(
    name="mesh_executor",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=5,
    code_execution_config={"work_dir": str(WORKSPACE), "use_docker": False}
)

solve_executor = autogen.UserProxyAgent(
    name="solve_executor",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=5,
    code_execution_config={"work_dir": str(WORKSPACE), "use_docker": False}
)

analysis_executor = autogen.UserProxyAgent(
    name="analysis_executor",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=3,
    code_execution_config={"work_dir": str(WORKSPACE), "use_docker": False}
)


# ========================================
# ORCHESTRATOR
# ========================================

class HybridFEMSystem:
    """Orchestrator with generic validation, rationale tracking, and DOLFINx compatibility"""
    
    def __init__(self):
        self.workspace = WORKSPACE
        self.decomposition = None

    def solve_problem(self, problem_description: str):
        print("HYBRID FEM SYSTEM (DOLFINx 0.10.0)")

        try:
            # Stage 0: DECOMPOSITION
            print("\nSTAGE 0: Decomposition")
            self._decompose(problem_description)

            # Stage 1: MESHING
            print("\nSTAGE 1: Meshing")
            if not self._execute_meshing():
                return {"success": False, "failed_at": "meshing"}
                
            # Stage 2: FORMULATION
            print("\nSTAGE 2: Formulation")
            if not self._execute_formulation():
                return {"success": False, "failed_at": "formulation"}
                
            # Stage 3: SOLVING
            print("\nSTAGE 3: Solving")
            if not self._execute_solving():
                return {"success": False, "failed_at": "solving"}

            # Stage 4: ANALYSIS
            print("\nSTAGE 4: Analysis")
            if not self._execute_analysis():
                return {"success": False, "failed_at": "analysis"}

            print("\n" + "="*60)
            print("✓ FEM Problem Solved Successfully!")
            print("="*60)
            
            self._print_decision_summary()

            return {"success": True, "outputs": self._collect_outputs()}
        
        except Exception as e:
            print(f"\n✗ ERROR: {e}")
            return {"success": False, "error": str(e)}

    def _decompose(self, problem_description: str):
        user_proxy = autogen.UserProxyAgent(
            name="user",
            human_input_mode="NEVER",
            max_consecutive_auto_reply=1,
            code_execution_config=None
        )

        user_proxy.initiate_chat(
            decomposer,
            message=f"Decompose this problem:\n\n{problem_description}"
        )

        response = user_proxy.chat_messages[decomposer][-1]["content"]

        if "```json" in response:
            json_str = response.split("```json")[1].split("```")[0]
        elif "```" in response:
            json_str = response.split("```")[1].split("```")[0]
        else:
            json_str = response
        
        self.decomposition = json.loads(json_str)
        print(f"  ✓ Decomposed into {len(self.decomposition['stages'])} stages")

    def _execute_meshing(self) -> bool:
        """Execute meshing with error feedback loop and DOLFINx compatibility"""
        
        stage = self.decomposition['stages'][0]
        
        # Build base message with ALL meshing info
        message_parts = [f"TASK: {stage['description']}"]
        
        if 'geometry_analysis' in stage:
            message_parts.append(f"\nGEOMETRY ANALYSIS:\n{json.dumps(stage['geometry_analysis'], indent=2)}")
        
        if 'strategy_files' in stage:
            message_parts.append(f"\nSTRATEGY FILES TO READ:\n{json.dumps(stage['strategy_files'], indent=2)}")
        
        if 'refinement_zones' in stage:
            message_parts.append(f"\nREFINEMENT: {stage['refinement_zones']}")
        
        message_parts.append("""
IMPORTANT:
1. First read the strategy files listed above
2. Document your approach BEFORE writing code
3. Follow the guidance in the strategy files
""")
        base_message = "\n\n".join(message_parts)
        
        # Error recovery loop
        last_error = None
        last_output = None
        
        for attempt in range(3):
            print(f"  - Meshing attempt {attempt+1}...")
            
            if attempt > 0 and last_error:
                error_feedback = f"""

    ⚠️ ATTEMPT {attempt} FAILED

    Error: {last_error}

    Output (truncated):
    {last_output[:800] if last_output else 'None'}

    """
                if attempt == 2:
                    error_feedback += """
    🔄 PARTITIONING FAILED TWICE. USE FALLBACK:
```python
    gmsh.option.setNumber("Mesh.Algorithm", 8)
    gmsh.option.setNumber("Mesh.RecombinationAlgorithm", 3)
    gmsh.option.setNumber("Mesh.RecombineAll", 1)
    gmsh.option.setNumber("Mesh.Smoothing", 100)
```

    Set meshing_method to "frontal_delaunay_fallback" in mesh_quality.json
    """
                
                current_message = base_message + error_feedback
            else:
                current_message = base_message
            
            try:
                mesh_executor.initiate_chat(meshing_specialist, message=current_message)
                
                last_output = self._get_last_execution_output(mesh_executor)
                
                success, issues = Validator.validate_mesh(self.workspace)
                
                if success:
                    # Validate mesh for DOLFINx
                    try:
                        self._ensure_dolfinx_compatible_mesh()
                    except Exception as e:
                        print(f"  ✗ Mesh compatibility error: {e}")
                        last_error = str(e)
                        continue
                    
                    with open(self.workspace / "mesh" / "mesh_quality.json") as f:
                        quality = json.load(f)
                    
                    print(f"  ✓ Success: {quality['num_elements']} {quality['element_type']} elements")
                    
                    rationale = quality.get('element_type_rationale', 'No rationale provided')
                    print(f"    💡 Rationale: {rationale}")
                    
                    return True
                else:
                    last_error = "\n".join(issues)
                    print(f"  ✗ Failed: {', '.join(issues)}")
                        
            except Exception as e:
                last_error = str(e)
                last_output = self._get_last_execution_output(mesh_executor)
                print(f"  ✗ Error: {e}")
        
        return False
    
    def _ensure_dolfinx_compatible_mesh(self):
        """Validate that mesh is DOLFINx-compatible."""
        from dolfinx.io import gmsh
        from mpi4py import MPI
        
        print("  🔧 Validating mesh for DOLFINx compatibility...")
        
        mesh_file = self.workspace / "mesh" / "mesh.msh"
        
        if not mesh_file.exists():
            raise RuntimeError("No mesh file found at mesh/mesh.msh")
        
        try:
            # DOLFINx 0.10.0 may return variable number of values
            result = gmsh.read_from_msh(str(mesh_file), MPI.COMM_WORLD, gdim=2)
            
            # Handle both 3-tuple and other return formats
            if isinstance(result, tuple):
                domain = result[0]
            else:
                domain = result
            
            cell_type = domain.topology.cell_name()
            num_cells = domain.topology.index_map(domain.topology.dim).size_local
            
            print(f"    ✅ Mesh validated: {num_cells} {cell_type} cells")
            print(f"    ✅ Mesh is ready for solver agent")
            
            return True
            
        except Exception as e:
            print(f"    ❌ Mesh validation error: {e}")
            raise RuntimeError(f"Mesh validation failed: {e}")

    def _execute_formulation(self) -> bool:
        """Execute formulation with validation and auto-fix"""
        
        stage = self.decomposition['stages'][1]
        
        message_parts = [f"TASK: {stage['description']}"]
        
        for key in stage.keys():
            if key not in ['stage_id', 'name', 'description', 'outputs']:
                message_parts.append(f"{key.upper()}: {json.dumps(stage[key], indent=2)}")
        
        message = "\n\n".join(message_parts)

        user_proxy = autogen.UserProxyAgent(
            name="user",
            human_input_mode="NEVER",
            max_consecutive_auto_reply=1,
            code_execution_config=None
        )

        user_proxy.initiate_chat(formulation_specialist, message=message)
        
        # Extract JSON from ALL messages
        messages = user_proxy.chat_messages[formulation_specialist]
        json_str = None
        config = None
        
        for msg in messages:
            content = msg.get("content", "")
            
            if "```json" in content:
                try:
                    json_str = content.split("```json")[1].split("```")[0].strip()
                    config = json.loads(json_str)
                    break
                except (IndexError, json.JSONDecodeError):
                    continue
            elif "```" in content:
                try:
                    json_str = content.split("```")[1].split("```")[0].strip()
                    config = json.loads(json_str)
                    break
                except (IndexError, json.JSONDecodeError):
                    continue
            else:
                try:
                    temp = content.strip()
                    if "TERMINATE" in temp:
                        terminate_idx = temp.rfind("TERMINATE")
                        temp = temp[:terminate_idx].strip()
                    config = json.loads(temp)
                    json_str = temp
                    break
                except json.JSONDecodeError:
                    continue
        
        if json_str is None:
            print("  ✗ Could not extract valid JSON from any agent message")
            return False
        
        # Write validated JSON
        config_file = self.workspace / "config" / "problem_config.json"
        try:
            with open(config_file, "w") as f:
                json.dump(config, f, indent=2)
            print(f"  ✓ Config file written ({len(json_str)} bytes)")
        except Exception as e:
            print(f"  ✗ Error writing config: {e}")
            return False
        
        # Validate
        valid, errors, warnings = Validator.validate_formulation_generic(self.workspace)
        
        if not valid:
            print(f"  ✗ Critical errors:")
            for err in errors:
                print(f"      - {err.get('msg', str(err))}")
            return False
        
        # Handle warnings
        if warnings:
            fixable_warnings = [w for w in warnings if w.get("fixable", False)]
            info_warnings = [w for w in warnings if not w.get("fixable", False)]
            
            if fixable_warnings:
                print(f"  ⚠️  Found {len(fixable_warnings)} auto-fixable issues:")
                for w in fixable_warnings:
                    print(f"      - {w['suggestion']}")
                
                fixed, fixes = Validator.auto_fix_formulation(self.workspace, fixable_warnings)
                
                if fixed:
                    print(f"  ✓ Auto-applied fixes:")
                    for fix in fixes:
                        print(f"      - {fix}")
            
            if info_warnings:
                print(f"  ℹ️  {len(info_warnings)} info messages:")
                for w in info_warnings:
                    print(f"      - {w['suggestion']}")
        
        # Display rationale
        try:
            with open(config_file) as f:
                config = json.load(f)
            rationale = config.get('formulation_rationale', 'No rationale provided')
            print(f"  ✓ Config created and validated")
            print(f"    💡 Rationale: {rationale}")
        except Exception:
            pass
        
        return True
    
    def _execute_solving(self) -> bool:
        """Execute solving with intelligent error diagnosis"""
        
        stage = self.decomposition['stages'][2]
        
        message_parts = [f"TASK: {stage['description']}"]
        
        if 'solver_guidance' in stage:
            message_parts.append(f"GUIDANCE: {json.dumps(stage['solver_guidance'], indent=2)}")
        
        message_parts.append("\nYOU DECIDE solver implementation details.")
        base_message = "\n\n".join(message_parts)
        
        last_error = None
        last_output = None
        
        for attempt in range(3):
            print(f"  Attempt {attempt + 1}/3...")
            
            if attempt > 0 and last_error:
                strategy = self._diagnose_solver_error(last_error, last_output)
                
                error_feedback = f"""

⚠️ PREVIOUS ATTEMPT {attempt} FAILED

Error details:
{last_error}

Console output:
{last_output[:2000] if last_output else 'No output'}

RECOVERY STRATEGY:
{strategy}

Please implement the fix described above. Generate complete corrected code.
"""
                current_message = base_message + error_feedback
            else:
                current_message = base_message
            
            try:
                solve_executor.initiate_chat(solver_specialist, message=current_message)
                
                last_output = self._get_last_execution_output(solve_executor)
                
                success, issues = Validator.validate_solution(self.workspace)
                
                if success:
                    with open(self.workspace / "solution" / "solver_metrics.json") as f:
                        metrics = json.load(f)
                    
                    rationale = metrics.get('solver_rationale', 'No rationale provided')
                    solver_type = metrics.get('solver_type', 'unknown')
                    element_deg = metrics.get('element_degree', 'unknown')
                    
                    print(f"  ✓ Success: converged={metrics.get('converged', False)}")
                    print(f"    ⚙️  Solver: {solver_type}, Element degree: {element_deg}")
                    print(f"    💡 Rationale: {rationale}")
                    
                    return True
                else:
                    last_error = "\n".join(issues)
                    print(f"  ✗ Failed: {', '.join(issues)}")
                        
            except Exception as e:
                last_error = str(e)
                last_output = self._get_last_execution_output(solve_executor)
                print(f"  ✗ Error: {e}")
        
        return False
    
    def _execute_analysis(self) -> bool:
        """Execute analysis with PNG output verification"""
        
        stage = self.decomposition['stages'][3]
        
        message_parts = [
            f"TASK: {stage['description']}",
            "",
            "CRITICAL REQUIREMENTS:",
            "1. Generate analysis report and save as JSON file",
            "2. Create summary.txt with human-readable findings",
            "3. All outputs must be saved in analysis/ directory",
            "",
            "EXPECTED OUTPUTS:",
            "- analysis/analysis_report.json",
            "- analysis/summary.txt"
        ]
        
        message = "\n".join(message_parts)
        
        last_error = None
        last_output = None
        
        for attempt in range(3):
            print(f"  Attempt {attempt+1}/3...")
            
            if attempt > 0 and last_error:
                error_feedback = f"""

⚠️ PREVIOUS ATTEMPT FAILED

Error: {last_error}

Output: {last_output[:1000] if last_output else 'No output'}

Please fix the code to address the specific error above.
"""
                current_message = message + error_feedback
            else:
                current_message = message
            
            try:
                analysis_executor.initiate_chat(analysis_specialist, message=current_message)
                
                last_output = self._get_last_execution_output(analysis_executor)
                
                # Check for required outputs
                report_file = self.workspace / "analysis" / "analysis_report.json"
                summary_file = self.workspace / "analysis" / "summary.txt"
                
                if report_file.exists():
                    print(f"  ✓ Analysis complete: {report_file.name}")
                    
                    try:
                        with open(report_file) as f:
                            report = json.load(f)
                        if 'conclusions' in report:
                            print(f"    📊 Conclusions: {report['conclusions'][:200]}...")
                    except Exception:
                        pass
                    
                    return True
                else:
                    last_error = "analysis_report.json not generated"
                    print(f"  ✗ {last_error}")
                    
            except Exception as e:
                last_error = str(e)
                last_output = self._get_last_execution_output(analysis_executor)
                print(f"  ✗ Error: {e}")
        
        return False
    
    def _get_last_execution_output(self, executor) -> str:
        """Extract stderr/stdout from last code execution"""
        try:
            agent_name = list(executor.chat_messages.keys())[0]
            messages = executor.chat_messages[agent_name]
            for msg in reversed(messages):
                if msg.get("role") == "user" and "exitcode" in msg.get("content", ""):
                    content = msg["content"]
                    if "Code output:" in content:
                        output = content.split("Code output:")[1].strip()
                        return output
            return "No execution output captured"
        except Exception as e:
            return f"Error extracting execution output: {e}"

    def _diagnose_solver_error(self, error: str, output: str) -> str:
        """Diagnose solver error and return recovery strategy"""
        
        error_lower = (error + " " + (output or "")).lower()
        
        if "petsc_options_prefix" in error_lower:
            return """
    ERROR TYPE: DOLFINx 0.10.0 API change - LinearProblem signature

    FIX: Add petsc_options_prefix parameter (must be non-empty string).

    problem = LinearProblem(
        a, L, bcs=bcs,
        petsc_options={"ksp_type": "preonly", "pc_type": "lu"},
        petsc_options_prefix="solve_"  # Non-empty prefix required
    )
    """
        
        if "has no attribute 'petsc'" in error_lower:
            return """
    ERROR TYPE: DOLFINx 0.10.0 import error

    FIX: Import petsc submodule explicitly.

    from dolfinx.fem.petsc import LinearProblem
    """
        
        if "degree of output function" in error_lower or "mesh degree" in error_lower:
            return """
    ERROR TYPE: XDMF write error - function degree mismatch

    FIX: Interpolate higher-degree functions to CG1 before writing to XDMF.

    CODE PATTERN:
    ```python
    # Create output space (degree 1 to match mesh)
    V_out = functionspace(domain, ("Lagrange", 1, (domain.geometry.dim,)))
    uh_out = Function(V_out)
    uh_out.interpolate(uh)

    with io.XDMFFile(comm, "solution/displacement.xdmf", "w") as xdmf:
        xdmf.write_mesh(domain)
        xdmf.write_function(uh_out)

    # Same for scalar stress
    W_out = functionspace(domain, ("Lagrange", 1))
    sigma_out = Function(W_out)
    expr = Expression(sigma_xx_expr, W_out.element.interpolation_points)
    sigma_out.interpolate(expr)

    with io.XDMFFile(comm, "solution/stress.xdmf", "w") as xdmf:
        xdmf.write_mesh(domain)
        xdmf.write_function(sigma_out)
    ```
    """

        if "all solution metrics are zero" in error_lower or "zero despite" in error_lower:
            return """
    ERROR TYPE: Zero solution despite forcing - likely config parsing issue

    FIX: The traction data may be in "tractions" list format, not "traction" simple format.

    Parse config like this:
    ```python
    # Parse tractions from list format
    tractions = cfg.get("tractions", [])
    traction_value = np.array([0.0, 0.0], dtype=default_scalar_type)
    traction_tag = 2  # right edge default

    if tractions:
        for t in tractions:
            region = t.get("region", "")
            value = t.get("value", [0.0, 0.0])
            if "right" in region.lower():
                traction_value = np.array(value, dtype=default_scalar_type)
                break
    else:
        traction_value = np.array(cfg.get("traction", [0.0, 0.0]), dtype=default_scalar_type)

    print(f"Traction applied: {traction_value}")
    ```

    Also verify boundary markers match expected tags and that traction is being applied to correct facets.
    """
        
        if "singular" in error_lower or "zero pivot" in error_lower:
            return """
    ERROR TYPE: Singular matrix - insufficient boundary conditions

    FIX: Add proper Dirichlet BCs to constrain all rigid body modes.
    For 2D elasticity, constrain at least:
    - One edge in x-direction
    - One point in y-direction (prevent rigid body rotation)
    """
        
        if "converge" in error_lower or "diverge" in error_lower:
            return """
    ERROR TYPE: Solver did not converge

    FIX: Use direct LU solver.

    problem = LinearProblem(
        a, L, bcs=bcs,
        petsc_options={"ksp_type": "preonly", "pc_type": "lu"},
        petsc_options_prefix="solve_"
    )
    """
        
        return f"""
    ERROR TYPE: Unclassified

    GENERAL STRATEGIES:
    1. Check imports are correct for DOLFINx 0.10.0
    2. Use direct solver (ksp_type: preonly, pc_type: lu)
    3. Interpolate functions to CG1 before XDMF write
    4. Check boundary conditions
    5. Verify config is being read correctly
    """
    
    def _print_decision_summary(self):
        """Print summary of all agent decisions"""
        
        print("\n" + "="*60)
        print("AGENT DECISION SUMMARY")
        print("="*60)
        
        try:
            with open(self.workspace / "mesh" / "mesh_quality.json") as f:
                mesh_info = json.load(f)
            print("\n📐 MESHING:")
            print(f"   Element type: {mesh_info.get('element_type', 'unknown')}")
            print(f"   Elements: {mesh_info.get('num_elements', 'unknown')}")
        except Exception:
            print("\n📐 MESHING: No info captured")
        
        try:
            with open(self.workspace / "config" / "problem_config.json") as f:
                config = json.load(f)
            print("\n📋 FORMULATION:")
            print(f"   Rationale: {config.get('formulation_rationale', 'N/A')}")
        except Exception:
            print("\n📋 FORMULATION: No rationale captured")
        
        try:
            with open(self.workspace / "solution" / "solver_metrics.json") as f:
                metrics = json.load(f)
            print("\n⚙️  SOLVING:")
            print(f"   Solver: {metrics.get('solver_type', 'unknown')}, Degree: {metrics.get('element_degree', 'unknown')}")
            print(f"   Converged: {metrics.get('converged', 'unknown')}")
        except Exception:
            print("\n⚙️  SOLVING: No rationale captured")
        
        try:
            with open(self.workspace / "analysis" / "analysis_report.json") as f:
                report = json.load(f)
            print("\n📊 ANALYSIS:")
            if 'conclusions' in report:
                print(f"   Conclusions: {report['conclusions'][:100]}...")
        except Exception:
            print("\n📊 ANALYSIS: No report captured")
        
        print("\n" + "="*60)

    def _collect_outputs(self):
        """Collect all output files"""
        outputs = {}
        for pattern in ["*.json", "*.png", "*.xdmf", "*.txt"]:
            for file in self.workspace.rglob(pattern):
                rel_path = str(file.relative_to(self.workspace))
                outputs[rel_path] = str(file)
        return outputs

In [2]:
if __name__ == "__main__":

    problem_description = """
A 200 mm-by-100 mm elastic plate contains a central circular hole of radius 20 mm. The material has Young's modulus of 200 GPa and Poisson's ratio of 0.25. 
The plate is modeled under plane-stress conditions. Make sure to use quadrilateral elements for meshing calculate the right amount of elements for best results.
The left edge (x = –100 mm) is fixed in the x direction, and the bottom-left corner is fixed in the y direction to prevent rigid-body motion.
A uniform tensile traction of 100 MPa is applied on the right edge (x = 100 mm) in the x direction, while the top, bottom, and hole boundaries are traction-free.
Solve for the displacement and stress field using FEniCS, compute σxx, and store the stress distribution result in a PNG file and also save a PNG file with the meshing that you did on the plate.
"""

# Run system
    system = HybridFEMSystem()
    result = system.solve_problem(problem_description)

    import json
    print(json.dumps(result, indent=2))  # Pretty print everything

HYBRID FEM SYSTEM (DOLFINx 0.10.0)

STAGE 0: Decomposition
user (to decomposer):

Decompose this problem:


A 200 mm-by-100 mm elastic plate contains a central circular hole of radius 20 mm. The material has Young's modulus of 200 GPa and Poisson's ratio of 0.25. 
The plate is modeled under plane-stress conditions. Make sure to use quadrilateral elements for meshing calculate the right amount of elements for best results.
The left edge (x = –100 mm) is fixed in the x direction, and the bottom-left corner is fixed in the y direction to prevent rigid-body motion.
A uniform tensile traction of 100 MPa is applied on the right edge (x = 100 mm) in the x direction, while the top, bottom, and hole boundaries are traction-free.
Solve for the displacement and stress field using FEniCS, compute σxx, and store the stress distribution result in a PNG file and also save a PNG file with the meshing that you did on the plate.


-------------------------------------------------------------------------